In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import seaborn as sns
import matplotlib as plt
%matplotlib inline
pd.pandas.set_option('display.max_rows', None)

from matplotlib import *
import sys
from pylab import *

import warnings
warnings.filterwarnings('ignore')

Data:
https://www.kaggle.com/datasets/abhinand05/magic-gamma-telescope-dataset

In [ ]:
df = pd.read_csv('/kaggle/input/magic-gamma-telescope-dataset/telescope_data.csv')

In [ ]:
df.head()

# Basic information about data set

In [ ]:
df.shape

In [ ]:
df.columns

In [ ]:
df.info()

**All the features have numerical data except the last one.**

In [ ]:
df.isnull().sum()

**No null values.**

In [ ]:
df['class'].value_counts()

~ 35% is hadron data and 65% is gamma data.. 

In [ ]:
df2 = df.copy()

In [ ]:
df2.head()

Converting the categorical variable.

In [ ]:
# we can label encode the class feature. 

"""
In this notebook we are denoting gamma event as '1' as that is our prime concern. 
As gamma is label encoded as '1' and we need to do a 
precise measurement for gamma, in this problem we will consider 'precision' as evaluation metric. 

One can also the other way around. In that case 'recall' should be the evaluation metric.
""" 

df2['class1'] = df2['class'].map({'g' : '1', 'h' : '0'})
df2['class1'] = df2['class1'].astype(int)
df2.head()

In [ ]:
data = df2.copy()

# dropping the first column 

data.drop('Unnamed: 0', axis = 1, inplace = True)
data.head()

In [ ]:
featlist = data.columns
featlist

# Basic EDA

In [ ]:
#checking the histograms for each feature
fig, axes = plt.subplots(3,4, figsize=(15,12) )
si1 = [[0,0], [0,1], [0,2], [0,3], [1,0], [1,1], [1,2], [1,3],  [2,0], [2,1], [2,2], [2,3]]

for item in range(len(featlist)-2): 
    i = si1[item][0]
    j = si1[item][1]
    sns.distplot(data[featlist[item]], kde = True, ax =axes[i,j])    
    
plt.show() 

In [ ]:
data.corr()

In [ ]:
sns.pairplot(data, hue = 'class', hue_order = ['h', 'g'])

**Observations:**  
We already know that this dataset has more gamma data than hadron data. This scenario is very much unlikely in real experiment. We will handle this behavior later. 

For now, looking at the pairplot, it is observed that the blue and 
yellow points are overlapped. So, we can try to proceed with Decision Trees and Ensemble techniques. If this does not work good, we can try upsampling/downsampling or SMOTE. 

In [ ]:
# let us explore all the features as a function of class1

fig, axes = plt.subplots(3,4, figsize=(16,12), tight_layout = True )
si1 = [[0,0], [0,1], [0,2], [0,3], 
       [1,0], [1,1], [1,2], [1,3], 
       [2,0], [2,1], [2,2], [2,3]]

for item in range(len(featlist)-2): 
    i = si1[item][0]
    j = si1[item][1]
    sns.histplot(data = data, hue = 'class1', x = featlist[item], ax =axes[i,j])
    axes[i,j].set_title(f'class vs {featlist[item]}')
    axes[i,j].set_xlabel('class')
    axes[i,j].set_ylabel(featlist[item])
    
    
plt.show()  

In [ ]:
plt.figure(figsize=(10,10))
sns.heatmap(data.corr(), annot = True)

Let us try to understand the data in detail. Let us try to study the correlation 
among the parameters for hadrons and gamma seperately. 

### Some more EDA for gamma and hadron seperately

In [ ]:
#data_gamma = data[[data['class1'] == 1]]
#data_gamma.head()

data_gamma = data[data['class1']==1]
data_gamma.shape

In [ ]:
sns.pairplot(data_gamma)

In [ ]:
figure(figsize = (8,8))
sns.heatmap(data_gamma.corr(), annot = True, cmap = 'RdYlGn')

In [ ]:
data_hadron = data[data['class1'] == 0]
data_hadron.shape

In [ ]:
sns.pairplot(data_hadron)

In [ ]:
figure(figsize = (10,10))
sns.heatmap(data_hadron.corr(), annot = True, cmap = 'RdYlGn')

# Model Building

- This problem in reality is a rare event detection. In the problem statement itself it was given that in this data set the gamma event are much more than the hadron event. This data set we have 65% gamma event and 35% hadron event.  But in practise, this is not the scenario.  The main aim is to detect gamma in a sea of hadrons. In this cases, it is very well established method to train the data on equal number of samples and then test on the test data. i.e, we will upsample the hadron data or downsample the gamma data so that there are same numbers of tagged data for hadrons and gamma. With these upsampled/downsampled data we will train the model and use the actual test data to predict and study the metric. Before performing upsampling/downsampling we would like to investige about which algorithm performs the task better.

- Let's start model building. We will go ahead with RandomForestClassifier, AdaBoost, GradientBoost and XgBoost .

In [ ]:
from sklearn.ensemble import RandomForestClassifier 
import xgboost as xgb 
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, roc_curve

In [ ]:
X = data[['fLength', 'fWidth', 'fSize', 'fConc', 'fConc1', 'fAsym', 'fM3Long',
       'fM3Trans', 'fAlpha', 'fDist']]
y = data[['class1']]

print(X.head(2))
print(y.head(2))
print(X.shape, y.shape)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)

In [ ]:
print(f'Shape of Training data : {X_train.shape}, {y_train.shape}')
print(f'Shape of Test data : {X_test.shape}{y_test.shape}')

In [ ]:
# first let's try with RandomForest Classifier: 

rf = RandomForestClassifier()
rf.fit(X_train, y_train)
pred_rf = rf.predict(X_test)
pred_prob_rf = rf.predict_proba(X_test)[:, 0]
pred_prob_rf_gamma = rf.predict_proba(X_test)[:, 1]

print(classification_report(y_test, pred_rf))
print(confusion_matrix(y_test, pred_rf))

In [ ]:
abc = AdaBoostClassifier()
abc.fit(X_train, y_train)
pred_abc = abc.predict(X_test)
pred_prob_abc = abc.predict_proba(X_test)[:, 0]
pred_prob_abc_gamma = abc.predict_proba(X_test)[:, 1] 

print(classification_report(y_test, pred_abc))
print(confusion_matrix(y_test, pred_abc))

In [ ]:
gbc = GradientBoostingClassifier()
gbc.fit(X_train, y_train)
pred_gbc = gbc.predict(X_test)
pred_prob_gbc = gbc.predict_proba(X_test)[:, 0]
pred_prob_gbc_gamma = gbc.predict_proba(X_test)[:, 1]

print(classification_report(y_test, pred_gbc))
print(confusion_matrix(y_test, pred_gbc))

In [ ]:
# let's try with XgBoost: 

xb = xgb.XGBRFClassifier()
xb.fit(X_train, y_train)
pred_xb = xb.predict(X_test)
pred_prob_xb = xb.predict_proba(X_test)[:,0]
pred_prob_xb_gamma = xb.predict_proba(X_test)[:, 1]

print(classification_report(y_test, pred_xb))
print(confusion_matrix(y_test, pred_xb))

### ROC curve :

In [ ]:
# plotting roc curve and finding auc score : 


# hadrons :
rf_fpr, rf_tpr, rf_threshold = roc_curve(y_test, pred_prob_rf, pos_label=0)
abc_fpr, abc_tpr, abc_threshold = roc_curve(y_test, pred_prob_abc, pos_label=0)
gbc_fpr, gbc_tpr, gbc_threshold = roc_curve(y_test, pred_prob_gbc, pos_label=0)
xb_fpr, xb_tpr, xb_threshold = roc_curve(y_test, pred_prob_xb, pos_label=0)


# gammas:
rf_fpr_g, rf_tpr_g, rf_threshold_g = roc_curve(y_test, pred_prob_rf_gamma, pos_label=1)
abc_fpr_g, abc_tpr_g, abc_threshold_g = roc_curve(y_test, pred_prob_abc_gamma, pos_label=1)
gbc_fpr_g, gbc_tpr_g, gbc_threshold_g = roc_curve(y_test, pred_prob_gbc_gamma, pos_label=1)
xb_fpr_g, xb_tpr_g, xb_threshold_g = roc_curve(y_test, pred_prob_xb_gamma, pos_label=1)

In [ ]:
plt.plot(rf_fpr, rf_tpr, color  = 'red', label='Random Forest Classifier')
plt.plot(abc_fpr, abc_tpr, color  = 'blue', label='AdaBoost Classifier')
plt.plot(gbc_fpr, gbc_tpr, color  = 'cyan', label = 'Gradient Boosting Classifier')
plt.plot(xb_fpr, xb_tpr, color  = 'green', label = 'XgBoost Classifier')
plt.title('Hadrons')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend(loc="lower right")
plt.grid()
plt.show()


plt.plot(rf_fpr_g, rf_tpr_g, color  = 'red', label='Random Forest Classifier')
plt.plot(abc_fpr_g, abc_tpr_g, color  = 'blue', label='AdaBoost Classifier')
plt.plot(gbc_fpr_g, gbc_tpr_g, color  = 'cyan', label = 'Gradient Boosting Classifier')
plt.plot(xb_fpr_g, xb_tpr_g, color  = 'green', label = 'XgBoost Classifier')
plt.title('Gammas')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend(loc="lower right")
plt.grid()
plt.show()


### AUC score:

In [ ]:
# auc score : 
auc_rf = roc_auc_score(y_test, pred_prob_rf_gamma)
auc_abc = roc_auc_score(y_test, pred_prob_abc_gamma)
auc_gbc = roc_auc_score(y_test, pred_prob_gbc_gamma)
auc_xb = roc_auc_score(y_test, pred_prob_xb_gamma)

print(f'Auc score for Random Forest Classifier : {round(auc_rf, 3)}')
print(f'Auc score for AdaBoost Classifier : {round(auc_abc, 3)}')
print(f'Auc score for Gradient Boosting : {round(auc_gbc, 3)}')
print(f'Auc score for XgBoost Classifier : {round(auc_xb, 3)}')


### Precision - Recall Curve: 


In [ ]:
from sklearn.metrics import precision_recall_curve

precision_rf1, recall_rf1, threshold_rf1 = precision_recall_curve(y_test, pred_prob_rf, pos_label = 0)
precision_rf0, recall_rf0, threshold_rf0 = precision_recall_curve(y_test, pred_prob_rf_gamma, pos_label = 1)


precision_abc1, recall_abc1, threshold_abc1 = precision_recall_curve(y_test, pred_prob_abc, pos_label = 0)
precision_abc0, recall_abc0, threshold_abc0 = precision_recall_curve(y_test, pred_prob_abc_gamma, pos_label = 1)

precision_gbc1, recall_gbc1, threshold_gbc1 = precision_recall_curve(y_test, pred_prob_gbc, pos_label = 0)
precision_gbc0, recall_gbc0, threshold_gbc0 = precision_recall_curve(y_test, pred_prob_gbc_gamma, pos_label = 1)

precision_xb1, recall_xb1, threshold_xb1 = precision_recall_curve(y_test, pred_prob_xb, pos_label = 0)
precision_xb0, recall_xb0, threshold_xb0 = precision_recall_curve(y_test, pred_prob_xb_gamma, pos_label = 1)

In [ ]:
plt.figure(figsize=(8,8))

plt.plot(precision_rf1, recall_rf1, color='red', label='RF - Hadrons')
plt.plot(precision_rf0, recall_rf0, color='red', linestyle = 'dashed' , label='RF - Gammas')

plt.plot(precision_abc1, recall_abc1, color='blue', label='Ada Boost - Hadrons')
plt.plot(precision_abc0, recall_abc0, color='blue', linestyle = 'dashed' , label='Ada Boost - Gammas')

plt.plot(precision_gbc1, recall_gbc1, color='cyan', label='Grad. Boosting - Hadrons')
plt.plot(precision_gbc0, recall_gbc0, color='cyan', linestyle = 'dashed' , label='Grad. Boosting - Gammas')

plt.plot(precision_xb1, recall_xb1, color='green', label='Xgboost - Hadrons')
plt.plot(precision_xb0, recall_xb0, color='green', linestyle = 'dashed' , label='Xgboost - Gammas')

plt.xlabel('Recall')
plt.ylabel('Precision')
plt.legend(loc="lower left")
plt.grid()
plt.show()

### Observations from different models:   
- In this problem our goal is to identify gammas as many as we can. In that case, 
all the above Models are working pretty good. The ROC curve and AUC scores are very good and RandomForest Classifier is the best among them. 

- For this particular problem, in the problem statement it is mentioned - " For technical reasons, the number of h events is underestimated. In the real data, the h class represents the majority of the events. The simple classification accuracy is not meaningful for this data, since classifying a background event as signal is worse than classifying a signal event as background. For comparison of different classifiers an ROC curve has to be used. The relevant points on this curve are those, where the probability of accepting a background event as signal is below one of the following thresholds: 0.01, 0.02, 0.05, 0.1, 0.2 depending on the required quality of the sample of the accepted events for different experiments."  -- This means that we need to supress/reject the background as much as we can. We cannot afford of mis-identifying a hadron as gamma. In this case we need to take care of 'false positive' outcomes and minimize it as far as possible. So, we should go with 'precision' as the metric for this problem.More is the precision, lower is the misidentification of false positives. if we compare the classification matrix for the models used, it is seen, that all of the models give a good precision. As a cost we are loosing signal which we can see from the 'recall' score. 

- Also, looking at the PR-curve, it can be observed that for recall > 0.9 and precision > 0.8, the gammas and hadrons are identified properly.  

This is the basic understanding this far. 
  
Let us try to improve the models as step by step discussed below.   
   
1. Let us perform Stratified K-Fold cross validation for RandomForest case and hypertune the parameters to improve the model. 
2. We can also try upsampling and compare the results and see if the model has improved. 

## Finding best models and best parameters using Cross validation and GridSearch

In [ ]:
from sklearn.model_selection import GridSearchCV

from sklearn.model_selection import StratifiedKFold

In [ ]:
# Hyperparameter tuning and cross validation for RandomForest model. 

rf1 = RandomForestClassifier()
params = {'n_estimators' : list(range(100, 500, 100)),
          'criterion' : ['gini', 'entropy']}

cv = StratifiedKFold(n_splits=5, random_state = None, shuffle = False)

clf=GridSearchCV(rf1, params, cv = cv,n_jobs=-1, scoring='recall')

In [ ]:
clf.fit(X_train,y_train)

In [ ]:
y_pred_rf_gs = clf.predict(X_test)
pred_prob_rf_gs = clf.predict_proba(X_test)[:, 0]
pred_prob_rf_gamma_gs =  clf.predict_proba(X_test)[:, 1]

In [ ]:
print(classification_report(y_test, y_pred_rf_gs))
print(confusion_matrix(y_test, y_pred_rf_gs))

In [ ]:
# lets find the ROC curve and AUC score for the above two models. 

#Next we would like to do random oversampling and see if the result improves.

In [ ]:
# plotting roc curve and finding auc score : 

rf_fpr_gs, rf_tpr_gs, rf_threshold_gs = roc_curve(y_test, pred_prob_rf_gamma_gs, pos_label=1)


In [ ]:
plt.plot(rf_fpr_gs, rf_tpr_gs, color  = 'red', label='Random Forest Classifier')

plt.title('ROC - After Grid Search')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend(loc="lower right")
plt.show()

In [ ]:
# auc score : 
auc_rf_gs = roc_auc_score(y_test, pred_prob_rf_gamma_gs)

print('After Grid Search')
print(f'Auc score for Random Forest Classifier : {round(auc_rf_gs, 3)}')

In [ ]:
# P-R after grid search : 

precision_rf1_gs, recall_rf1_gs, threshold_rf1_gs = precision_recall_curve(y_test, pred_prob_rf_gs, pos_label = 0)
precision_rf0_gs, recall_rf0_gs, threshold_rf0_gs = precision_recall_curve(y_test, pred_prob_rf_gamma_gs, pos_label = 1)

In [ ]:
plt.figure(figsize=(8,8))

plt.plot(precision_rf1_gs, recall_rf1_gs, color='red', label='RF - Hadrons')
plt.plot(precision_rf0_gs, recall_rf0_gs, color='red', linestyle = 'dashed' , label='RF - Gammas')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.legend(loc="lower left")
plt.grid()
plt.show()

**Comment:** 

Using GridSearch the result didn't improve significantly. 


## Next... 

- We found that the RandomForestClassifier perfoms best. Now we need to train the model with equal numbers of hadrons and gammas and then test on the test data as we have discussed earlier.

- The very usual approach is to train the model using same number of hadron and gamma data and predict the test data. So, in this case we will upsample the hadron data or downsample the gamma data so that there are same number of tagged data for hadrons and gamma. With these upsampled/downsampled data we will train the model and use the actual test data to predict and study the metric.

- In this particular notebook we will perform oversampling of data and will use RandomForestClassifier model for prediction. 
(Downsampling is not done, as we loose the data information)

### Oversampling :

In [ ]:
from imblearn.over_sampling import RandomOverSampler
from collections import Counter

In [ ]:
os = RandomOverSampler(sampling_strategy = 'minority')
X_train_ns,y_train_ns = os.fit_resample(X_train,y_train)
print("The number of classes before fit {}".format(y_train.value_counts()))
print("The number of classes after fit {}".format(y_train_ns.value_counts()))

In [ ]:
y_test.value_counts()

In [ ]:
rf1_ns = RandomForestClassifier()
params_ns = {'n_estimators' : list(range(100, 500, 100)),
          'criterion' : ['gini', 'entropy']}

cv_ns = StratifiedKFold(n_splits=5, random_state = None, shuffle = False)

clf_ns=GridSearchCV(rf1_ns, params_ns, cv = cv_ns,n_jobs=-1, scoring='precision')


#rf_os = RandomForestClassifier()
#rf_os.fit(X_train_ns, y_train_ns)

clf_ns.fit(X_train_ns, y_train_ns)
pred_rf_os = clf_ns.predict(X_test)
pred_prob_rf_os = clf_ns.predict_proba(X_test)[:, 1]

print(classification_report(y_test, pred_rf_os))
print(confusion_matrix(y_test, pred_rf_os))

**ROC curve and AUC score:** 



In [ ]:
# roc curve
rf_fpr_ns, rf_tpr_ns, rf_threshold_ns = roc_curve(y_test, pred_prob_rf_os, pos_label=1)
plt.plot(rf_fpr_ns, rf_tpr_ns, color  = 'red', label='Random Forest Classifier')

plt.title('ROC - RandomOverSampler')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend(loc="lower right")
plt.show()

In [ ]:
# auc score : 
auc_rf_gs = roc_auc_score(y_test, pred_prob_rf_os)

print('After RandomOverSampler')
print(f'Auc score for Random Forest Classifier : {round(auc_rf_gs, 3)}')

### SMOTEK :

In [ ]:
from imblearn.combine import SMOTETomek

In [ ]:
sm = SMOTETomek(sampling_strategy = 'minority')
X_train_sm,y_train_sm = sm.fit_resample(X_train,y_train)
print("The number of classes before fit {}".format(y_train.value_counts()))
print("The number of classes after fit {}".format(y_train_sm.value_counts()))

In [ ]:
rf1_sm = RandomForestClassifier()
params_sm = {'n_estimators' : list(range(100, 500, 100)),
          'criterion' : ['gini', 'entropy']}

cv_sm = StratifiedKFold(n_splits=5, random_state = None, shuffle = False)

clf_sm = GridSearchCV(rf1_sm, params_sm, cv = cv_sm,n_jobs=-1, scoring='precision')


#rf_os = RandomForestClassifier()
#rf_os.fit(X_train_ns, y_train_ns)
y_train_sm1 = np.ravel(y_train_sm)
clf_sm.fit(X_train_sm, y_train_sm1)
pred_rf_sm = clf_sm.predict(X_test)
pred_prob_rf_sm = clf_sm.predict_proba(X_test)[:, 1]

print(classification_report(y_test, pred_rf_sm))
print(confusion_matrix(y_test, pred_rf_sm))

**ROC curve and AUC score :**

In [ ]:
# roc curve
rf_fpr_sm, rf_tpr_sm, rf_threshold_sm = roc_curve(y_test, pred_prob_rf_sm, pos_label=1)
plt.plot(rf_fpr_sm, rf_tpr_sm, color  = 'red', label='Random Forest Classifier')

plt.title('ROC - SMOTETomek')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend(loc="lower right")
plt.show()

In [ ]:
# auc score : 
auc_rf_gs = roc_auc_score(y_test, pred_prob_rf_sm)

print('After SMOTETomek')
print(f'Auc score for Random Forest Classifier : {round(auc_rf_gs, 3)}')

## Threshold Tuning : 
Once we have the model and understood the ROC and PR curve, the next job is to find the optimum value of the threshold which leads to the best precision-recall trade-off. Mostly this incorporates the knowledge of domain expert. 
But in this notebook we will just go with the flow. As we have discussed earlier, that for this problem precision is very important and hence we will tune the threshold to obtain maximum precision without loosing recall. It is observed that though the AUC scores for the Random Forest Grid Search Model, Oversampler and SMOTETomek are nearly equal but the classification metric is better for the SMOTETomek model. So, in this notebook we will do this step for the 
SMOTETomek model. 

Two common approaches will be perfomed: 
1. G mean 
2. Youden's statistic

**Threshold for ROC curve :**

In [ ]:
# using Gmean : 
# finding out max gmean and the corresponding threshold and tpr and fpr

def get_gmean_threshold_fpr_tpr(tpr, fpr, threshold):
    gmean  = np.sqrt(tpr* (1 - fpr))
    opt_gmean = np.max(gmean)
    opt_gmean_index = np.argmax(gmean)
    #opt_mean_index = np.where(opt_gmean)
    opt_threshold = threshold[opt_gmean_index]
    opt_fpr = round(fpr[opt_gmean_index], 5)
    opt_tpr = round(tpr[opt_gmean_index], 5)
    
    return gmean, opt_gmean, opt_gmean_index, opt_threshold, opt_fpr, opt_tpr

gmean, opt_gmean, opt_gmean_index,opt_threshold, opt_fpr, opt_tpr  = get_gmean_threshold_fpr_tpr(rf_tpr_sm, rf_fpr_sm, rf_threshold_sm)
print(f'Optimum gmean: {opt_gmean}')
print(f'Index for optimum mean : {opt_gmean_index}')
print(f'Threshold corresponding to Optimum Gmean :{opt_threshold}') 
print(f'FPR corresponding to Optimum Gmean :  {opt_fpr}')
print(f'TPR corresponding to Optimum Gmean : {opt_tpr}')

In [ ]:
# plotting : 

fig, axes = plt.subplots(1,2, figsize=(8,3), tight_layout = True )

axes[0].plot(rf_threshold_sm, gmean, color = 'blue')
axes[0].plot(opt_threshold, opt_gmean, color = 'black', 
         marker = ".", markersize = 12)
axes[0].set_title('Gmean vs threshold values')
axes[0].set_xlabel('Threshold values')
axes[0].set_ylabel('Gmean')
#axes[0].show()



axes[1].plot(rf_fpr_gs, rf_tpr_gs, color = 'red')
axes[1].plot(opt_fpr, opt_tpr, color = 'black', marker = '*', 
         markersize = 12, label = 'RandomForestClassifier')

axes[1].set_title('ROC - SMOTETomek with threshold values')
axes[1].set_xlabel('False Positive Rate')
axes[1].set_ylabel('True Positive Rate')
axes[1].legend(loc="lower right")
#axes[1].show()

plt.show()

**Inference**   
With is exercise we first find the optimum threshold value for this problem from gmean vs threshold data (shown by 'dot' on the LHS plot). The optimum value for 
threshold is 0.58. Next we try to find out the optimum tpr and fpr values corresponding to the optimum threshold value = 0.58. This is shown by the 'star' marker on the RHS plot. The other values are listed above.   
So given this problem, after calculating the probabilities of outputs, one can use threshold value as 0.58 and find out the two classes. We expect that these choices will lead to the best predictions.

In [ ]:
# using Youden's statistic

def get_ystat_threshold_fpr_tpr(tpr, fpr, threshold):
    ystat  = tpr - fpr
    opt_ystat = np.max(ystat)
    opt_ystat_index = np.argmax(ystat)
    #opt_ystat_index = np.where(opt_ystat)
    opt_threshold = threshold[opt_ystat_index]
    opt_fpr = fpr[opt_ystat_index]
    opt_tpr = tpr[opt_ystat_index]
    
    return ystat, opt_ystat, opt_ystat_index, opt_threshold, opt_fpr, opt_tpr

ystat, opt_ystat, opt_ystat_index,opt_threshold, opt_fpr, opt_tpr  = get_ystat_threshold_fpr_tpr(rf_tpr_sm, rf_fpr_sm, rf_threshold_sm)
print(f'Optimum ystat: {opt_ystat}')
print(f'Index for optimum mean : {opt_ystat_index}')
print(f'Threshold corresponding to Optimum Gmean :{opt_threshold}') 
print(f'FPR corresponding to Optimum Gmean :  {opt_fpr}')
print(f'TPR corresponding to Optimum Gmean : {opt_tpr}')



## Results after threshold tuning : 
Now,  we need to plugin this threshold value for the prediction and find out whether the prediction statistics improve or not.

In [ ]:
#y_pred_new = (clf.predict_proba(X_test)[:,1] >= opt_threshold).astype(bool) # set threshold as 0.3
y_pred_new = (pred_prob_rf_gamma_gs >= opt_threshold).astype(bool)
y_pred_new

In [ ]:
print(classification_report(y_test, np.ravel(y_pred_new)))
print(confusion_matrix(y_test, np.ravel(y_pred_new)))

If we take a look at the confusion matrix, we can observe that number of misidentifications are reduced. The metrics have improved a bit in comparison to the SMOTETomek model but they have improved significantly in comparison to simple random forest classifer or adaboost or random forest with grid search.

## Conclusions :
In this project, the main aim was to identify gamma in a haystack of hadrons. The dataset contains more gamma data than hadron data. Looking at the pairplot we performed the classification task using the Ensemble techniques. Looking at the ROC curve, AUC score and PR curve we can say that RandomForest Classifier did the best job. The hyperparameter tuning was also performed but the result did not improve much. As there were a difference in number of hadron and gamma data, the upsampling technique was also perfomed using RandomOverSampler and SmotekTomek. The AUC scores have not improve much from the initial one, though if we take a look at the precision-recall values, SMOTETomek has the best one. 
Next we also performed the threshold tuning technique to improve predictions. As 
the metric for SMOTETomek is the best among all the models discussed, we decided to perform the threshold tuning for this model. We obtained the optimum threshold for this case and using this threshold value we obtain predictions. This time the metrics and the confusion matrix we get is better than what we had earlier.  

**References :**

- https://machinelearningmastery.com/tune-learning-rate-for-gradient-boosting-with-xgboost-in-python/   

- https://inria.github.io/scikit-learn-mooc/python_scripts/ensemble_hist_gradient_boosting.html  

- https://www.analyticsvidhya.com/blog/2016/02/complete-guide-parameter-tuning-gradient-boosting-gbm-python/   

- https://medium.com/@chaudhurysrijani/tuning-of-adaboost-with-computational-complexity-8727d01a9d20    

- https://github.com/krishnaik06/Handle-Imbalanced-Dataset/blob/master/handling-imbalanced.ipynb    

- https://www.kaggle.com/general/262007  

- https://www.kaggle.com/code/rafjaa/resampling-strategies-for-imbalanced-datasets  

- https://www.kaggle.com/code/gargmanish/how-to-handle-imbalance-data-study-in-detail/notebook  


-  https://neptune.ai/blog/evaluation-metrics-binary-classification 

- ROC curve
 - https://waterprogramming.wordpress.com/2021/02/08/how-do-we-deal-with-extreme-events-and-imbalanced-datasets-in-machine-learning/ 

 - https://www.displayr.com/what-is-a-roc-curve-how-to-interpret-it/ 

 - https://towardsdatascience.com/understanding-auc-roc-curve-68b2303cc9c5  

 - https://www.analyticsvidhya.com/blog/2020/06/auc-roc-curve-machine-learning/  

 - https://developers.google.com/machine-learning/crash-course/classification/roc-and-auc   

 - https://www.statology.org/interpret-roc-curve/  


- Precision Recall curve 

 - https://scikit-learn.org/stable/auto_examples/model_selection/plot_precision_recall.html#:~:text=The%20precision%2Drecall%20curve%20shows,a%20low%20false%20negative%20rate. 

 - https://medium.com/@douglaspsteen/precision-recall-curves-d32e5b290248  

 - https://www.datascienceblog.net/post/machine-learning/interpreting-roc-curves-auc/  

 - https://analyticsindiamag.com/complete-guide-to-understanding-precision-and-recall-curves/  

 - https://www.geeksforgeeks.org/precision-recall-curve-ml/ 

 - https://machinelearningmastery.com/roc-curves-and-precision-recall-curves-for-classification-in-python/  

 - https://machinelearningmastery.com/roc-curves-and-precision-recall-curves-for-imbalanced-classification/  

 - https://acutecaretesting.org/en/articles/precision-recall-curves-what-are-they-and-how-are-they-used  
 

- Threshold tuning 

 - https://towardsdatascience.com/optimal-threshold-for-imbalanced-classification-5884e870c293
 
 - https://machinelearningmastery.com/threshold-moving-for-imbalanced-classification/

 - https://towardsdatascience.com/finding-the-best-classification-threshold-for-imbalanced-classifications-with-interactive-plots-7d65828dda38 

 - https://www.yourdatateacher.com/2021/06/14/are-you-still-using-0-5-as-a-threshold/ 

 - https://stats.stackexchange.com/questions/312119/reduce-classification-probability-threshold 

 - https://pub.towardsai.net/improve-your-classification-models-with-threshold-tuning-bb69fca15114
